In [153]:
from urllib.request import urlopen
import requests
import bs4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd

In [ ]:
url = "https://music.bugs.co.kr/chart"
try:
    response = requests.get(url)
    print("벅스 접속 성공!")
except Exception as e:
    print(f'벅스 접속 실패 : {e}')


벅스 접속 성공!


In [155]:
url2 = "https://www.melon.com/chart/index.htm"
try:
    response2 = requests.get(url2)
    print("멜론 접속 성공!")
except Exception as e:
    print(f'멜론 접속 실패 : {e}')

멜론 접속 성공!


In [156]:
url3 = "https://www.genie.co.kr/chart/top200"
try:
    response3 = requests.get(url3)
    print("지니 접속 성공!")
except Exception as e:
    print(f'지니 접속 실패 : {e}')

지니 접속 성공!


In [174]:
bugs = pd.DataFrame(columns=["서비스", "순위", "타이틀", "가수"])
melon = pd.DataFrame(columns=["서비스", "순위", "타이틀", "가수"])
genie = pd.DataFrame(columns=["서비스", "순위", "타이틀", "가수"])

In [158]:
res = requests.get(url, stream=True)
html = res.content
bs_b = bs4.BeautifulSoup(html,'html.parser')

In [159]:
def get_bugs_chart():
    titles = bs_b.select('p.title a')
    artists = bs_b.select('p.artist')
    for title, artist in zip(titles[:100],artists[:100]):
        bugs.loc[len(bugs)] = {
            "서비스" : "Bugs",
            "순위" : len(bugs) + 1,
            "타이틀" : title.text.strip(),
            "가수" : artist.find_all('a')[0].text.strip(),
        }
get_bugs_chart()

In [160]:
bugs

,서비스,순위,타이틀,가수
0,Bugs,1,REBEL HEART,IVE (아이브)
1,Bugs,2,Whiplash,aespa
2,Bugs,3,"HOME SWEET HOME (feat. 태양, 대성)",G-DRAGON
3,Bugs,4,toxic till the end,로제(ROSÉ)
4,Bugs,5,APT.,로제(ROSÉ)
...,...,...,...,...
95,Bugs,96,stay a little longer,로제(ROSÉ)
96,Bugs,97,우연 (우울한 우연),베이비복스(Baby V.O.X)
97,Bugs,98,시작의 아이,마크툽(MAKTUB)
98,Bugs,99,Take Care,다이나믹 듀오


In [161]:
# 멜론과 지니는 해당 방식을 이용하기 위해 header 값이 필요
melon_header = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36'}

In [162]:
def get_melon_chart():
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.get(url2)
    titles = driver.find_elements(By.CSS_SELECTOR, 'div.ellipsis.rank01 a')
    artists = driver.find_elements(By.CSS_SELECTOR, 'div.ellipsis.rank02')
    for title, artist in zip(titles[:100],artists[:100]):
        melon.loc[len(melon)] = {
            "서비스" : "Melon",
            "순위" : len(melon) + 1,
            "타이틀" : title.text.strip(),
            "가수" : artist.find_element(By.CSS_SELECTOR, 'a').text.strip(),
        }
    driver.quit()
get_melon_chart()

In [163]:
melon

,서비스,순위,타이틀,가수
0,Melon,1,"HOME SWEET HOME (feat. 태양, 대성)",G-DRAGON
1,Melon,2,Whiplash,aespa
2,Melon,3,APT.,로제 (ROSÉ)
3,Melon,4,나는 반딧불,황가람
4,Melon,5,REBEL HEART,IVE (아이브)
...,...,...,...,...
95,Melon,96,Dynamite,방탄소년단
96,Melon,97,봄날,방탄소년단
97,Melon,98,네모네모,YENA (최예나)
98,Melon,99,파이팅 해야지 (Feat. 이영지),부석순 (SEVENTEEN)


In [164]:
def get_genie_chart(page):
    url = "https://www.genie.co.kr/chart/top200?ditc=D&ymd=20250123&hh=14&rtm=Y&pg=" + str(page)
    genie_header = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36'}
    res = requests.get(url, stream=True, headers=genie_header)
    html = res.content
    bs_g = bs4.BeautifulSoup(html,'html.parser')
    titles = bs_g.select('td.info .title.ellipsis')
    artists = bs_g.select('td.info .artist.ellipsis')
    for title, artist in zip(titles[:100],artists[:100]):
        genie.loc[len(genie)] = {
                "서비스" : "Genie",
                "순위" : len(genie) + 1,
                "타이틀" : title.text.strip(),
                "가수" : artist.text.strip(),
            }
for page in range(1,3):
    get_genie_chart(page)

In [165]:
genie

,서비스,순위,타이틀,가수
0,Genie,1,HOME SWEET HOME (Feat. 태양 & 대성),G-DRAGON
1,Genie,2,Whiplash,aespa
2,Genie,3,APT.,로제 (ROSÉ) & Bruno Mars
3,Genie,4,REBEL HEART,IVE (아이브)
4,Genie,5,나는 반딧불,황가람
...,...,...,...,...
95,Genie,96,Polaroid,임영웅
96,Genie,97,시작의 아이,마크툽 (Maktub)
97,Genie,98,보금자리,임영웅
98,Genie,99,인생찬가,임영웅


In [166]:
final_df = pd.concat([melon,bugs,genie],axis=0)
final_df

,서비스,순위,타이틀,가수
0,Melon,1,"HOME SWEET HOME (feat. 태양, 대성)",G-DRAGON
1,Melon,2,Whiplash,aespa
2,Melon,3,APT.,로제 (ROSÉ)
3,Melon,4,나는 반딧불,황가람
4,Melon,5,REBEL HEART,IVE (아이브)
...,...,...,...,...
95,Genie,96,Polaroid,임영웅
96,Genie,97,시작의 아이,마크툽 (Maktub)
97,Genie,98,보금자리,임영웅
98,Genie,99,인생찬가,임영웅


In [168]:
final_df.to_excel("MP4.xlsx", index=False)

In [ ]:
melon_header = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36'}

In [172]:
def get_melon_chart2():
    url = "https://www.melon.com/chart/index.htm"
    header = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36'}
    res = requests.get(url, stream=True, headers=header)
    html = res.content
    bs = bs4.BeautifulSoup(html,'html.parser')
    titles = bs.select('div.ellipsis.rank01 a')
    artists = bs.select('div.ellipsis.rank02')
    for title, artist in zip(titles[:100],artists[:100]):
        melon.loc[len(melon)] = {
            "서비스" : "Melon",
            "순위" : len(melon) + 1,
            "타이틀" : title.text.strip(),
            "가수" : artist.find_all('a')[0].text.strip(),
        }

def get_bugs_chart():
    url = "https://music.bugs.co.kr/chart"
    header = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36'}
    res = requests.get(url, stream=True, headers=header)
    html = res.content
    bs = bs4.BeautifulSoup(html,'html.parser')
    titles = bs.select('p.title a')
    artists = bs.select('p.artist')
    for title, artist in zip(titles[:100],artists[:100]):
        bugs.loc[len(bugs)] = {
            "서비스" : "Bugs",
            "순위" : len(bugs) + 1,
            "타이틀" : title.text.strip(),
            "가수" : artist.find_all('a')[0].text.strip(),
        }

def get_genie_chart(page):
    url = "https://www.genie.co.kr/chart/top200?ditc=D&ymd=20250123&hh=14&rtm=Y&pg=" + str(page)
    header = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36'}
    res = requests.get(url, stream=True, headers=header)
    html = res.content
    bs = bs4.BeautifulSoup(html,'html.parser')
    titles = bs.select('td.info .title.ellipsis')
    artists = bs.select('td.info')
    for title, artist in zip(titles[:100],artists[:100]):
        genie.loc[len(genie)] = {
                "서비스" : "Genie",
                "순위" : len(genie) + 1,
                "타이틀" : title.text.strip(),
                "가수" : artist.find_all('a')[1].text.strip(),
            }



In [ ]:
# url, header, titles, artists, 서비스, df, 

In [182]:
import requests
import bs4
import pandas as pd

def get_music_charts():
    # 결과를 담을 데이터프레임
    final_df = pd.DataFrame(columns=["서비스", "순위", "타이틀", "가수"])

    # 공통 헤더
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36'}

    # 서비스별 설정
    services = {
        "Melon": {
            "url": "https://www.melon.com/chart/index.htm",
            "title_selector": "div.ellipsis.rank01 a",
            "artist_selector": "div.ellipsis.rank02",
            "artist_tag_index": 0,
        },
        "Bugs": {
            "url": "https://music.bugs.co.kr/chart",
            "title_selector": "p.title a",
            "artist_selector": "p.artist",
            "artist_tag_index": 0,
        },
        "Genie": {
            "url_template": "https://www.genie.co.kr/chart/top200?ditc=D&ymd=20250123&hh=14&rtm=Y&pg={page}",
            "title_selector": "td.info .title.ellipsis",
            "artist_selector": "td.info",
            "artist_tag_index": 1,
            "pages": 2,
        },
    }

    # 차트 데이터 수집 함수
    def fetch_chart_data(service_name, config):
        service_df = pd.DataFrame(columns=["서비스", "순위", "타이틀", "가수"])

        if service_name == "Genie":
            for page in range(1, config["pages"] + 1):
                url = config["url_template"].format(page=page)
                res = requests.get(url, stream=True, headers=header)
                html = res.content
                bs = bs4.BeautifulSoup(html, 'html.parser')
                titles = bs.select(config["title_selector"])
                artists = bs.select(config["artist_selector"])

                for i, (title, artist) in enumerate(zip(titles, artists)):
                    try:
                        service_df.loc[len(service_df)] = {
                            "서비스": service_name,
                            "순위": len(service_df) + 1,
                            "타이틀": title.text.strip(),
                            "가수": artist.find_all('a')[config["artist_tag_index"]].text.strip(),
                        }
                    except IndexError:
                        service_df.loc[len(service_df)] = {
                            "서비스": service_name,
                            "순위": len(service_df) + 1,
                            "타이틀": title.text.strip(),
                            "가수": "Unknown",
                        }
        else:
            res = requests.get(config["url"], stream=True, headers=header)
            html = res.content
            bs = bs4.BeautifulSoup(html, 'html.parser')
            titles = bs.select(config["title_selector"])
            artists = bs.select(config["artist_selector"])

            for i, (title, artist) in enumerate(zip(titles, artists)):
                try:
                    service_df.loc[len(service_df)] = {
                        "서비스": service_name,
                        "순위": len(service_df) + 1,
                        "타이틀": title.text.strip(),
                        "가수": artist.find_all('a')[config["artist_tag_index"]].text.strip(),
                    }
                except IndexError:
                    service_df.loc[len(service_df)] = {
                        "서비스": service_name,
                        "순위": len(service_df) + 1,
                        "타이틀": title.text.strip(),
                        "가수": "Unknown",
                    }

        return service_df

    # 각 서비스의 차트 데이터를 합치기
    for service_name, config in services.items():
        final_df = pd.concat([final_df, fetch_chart_data(service_name, config)], ignore_index=True)

    return final_df

# 함수 실행 후 결과 저장
final_df = get_music_charts()
# final_df.to_excel("Music_Chart.xlsx", index=False)
# print("Music chart data saved to Music_Chart.xlsx")


In [ ]:
def get_melon_chart2():
    url = "https://www.melon.com/chart/index.htm"
    header = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36'}
    res = requests.get(url, stream=True, headers=header)
    html = res.content
    bs = bs4.BeautifulSoup(html,'html.parser')

    print(bs)

    titles = bs.select('div.ellipsis.rank01 a')
    artists = bs.select('div.ellipsis.rank02')
    for title, artist in zip(titles[:100],artists[:100]):
        melon.loc[len(melon)] = {
            "서비스" : "Melon",
            "순위" : len(melon) + 1,
            "타이틀" : title.text.strip(),
            "가수" : artist.find_all('a')[0].text.strip(),
        }

In [185]:
url = "https://www.melon.com/chart/index.htm"
header = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36'}
res = requests.get(url, stream=True, headers=header)
html = res.content
bs = bs4.BeautifulSoup(html,'html.parser')

print(bs)

<!DOCTYPE html>

<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title>멜론차트&gt;TOP100&gt;멜론</title>
<meta content="음악서비스, 멜론차트, 멜론TOP100, 최신음악, 인기가요, 뮤직비디오, 앨범, 플레이어, 스트리밍, 다운로드, 아티스트플러스, 아티스트채널" name="keywords"/>
<meta content="No.1 뮤직플랫폼 멜론! 최신 트렌드부터 나를 아는 똑똑한 음악추천까지!" name="description"/>
<meta content="ee85ff6db1fa8f2226bcb671ecb2bcdbcffb6f8b" name="naver-site-verification"/>
<meta content="q4tbTQhmxa4La3OdNLsNOCxrJ_WV6lUlBFrFW4-HqQc" name="google-site-verification"/>
<meta content="357952407588971" property="fb:app_id"/>
<meta content="Melon" property="og:title"/>
<meta content="https://cdnimg.melon.co.kr/resource/image/web/common/logo_melon142x99.png" property="og:image"/>
<meta content="음악이 필요한 순간, 멜론" property="og:description"/>
<meta content="http://www.melon.com/chart/index.htm" property="og:url"/>
<meta content="website" property="og:type"/>
<meta content="멜론" proper